In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint

In [2]:
df = pd.read_csv("data/baseline_cases_runs.csv")
reference = "nawi"

In [ ]:
for source_water in ["industrial"]: #df.source_water_category.unique():
    for case_study in df[df.source_water_category == source_water].case_study.unique():
        h1 = df[df.case_study == case_study]
        scenarios = h1.scenario.unique()
#         if case_study in ["upw"]:
#             scenarios = ["zld_a", "zld_b", "zld_c", "baseline"]
#         else:        
#             scenarios = h1.scenario.unique()

        for scenario in scenarios:

            print("------------------------------------------------------")
            print("------------------------------------------------------")
            print("-----------", case_study, scenario, "-----------------")
            print("------------------------------------------------------")
            print("------------------------------------------------------")

            desired_recovery = df[((df.case_study == case_study) & (df.scenario == scenario))].max_recovery_rate.max()

            source_water_category = df[((df.case_study == case_study) & (df.scenario == scenario))].source_water_category.max()

            skip_small_base = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_base.max()
            skip_small_sens = df[((df.case_study == case_study) & (df.scenario == scenario))].skip_small_sens.max()
            ro_bounds = df[((df.case_study == case_study) & (df.scenario == scenario))].ro_bounds.max()

            m = wt.watertap_setup(dynamic=False, case_study = case_study, reference = reference, 
                                  scenario = scenario, source_scenario = "baseline")

            m = wt.case_study_trains.get_case_study(m=m)

            for key in m.fs.pfd_dict.keys():
                if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
                    getattr(m.fs, key).a.fix(4.2)
                    getattr(m.fs, key).b.fix(0.35)

            if "1p5" in scenario:
                m.fs.irrigation_and_drainage.flow_vol_in.fix(0.0657)
            
            ############################################################
            ############################################################
            # set recovery for uranium case RO units.
            if case_study == "uranium":
                m.fs.ion_exchange.removal_fraction[0, "tds"].unfix()
                m.fs.ion_exchange.water_recovery.fix(.66)
                m.fs.ion_exchange.anion_res_capacity.unfix()
                m.fs.ion_exchange.cation_res_capacity.unfix()
            
            if scenario == "swd_well":
                m.fs.produced_water_for_dw.conc_mass_in[0, "tds"].fix(50)
            
            # run and and return baseline with 0 degrees of freedom
            m = wt.run_water_tap_ro(m, source_water_category = source_water_category, 
                                 desired_recovery = desired_recovery, skip_small = skip_small_base, 
                                 ro_bounds = ro_bounds, source_scenario = "baseline", scenario_name = scenario)

            if scenario == "baseline":
                wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)

                if source_water == "seawater":
                    m = wt.get_fixed_onm_reduction(m, reduction_value_list = [0.9, 0.8, 0.7, 0.6, 0.5],
                                                  skip_small=skip_small_base)

                if case_study == "san_luis":
                    n = 1
                    for flow_scenario in [0.0657, 0.1314, 0.2191]:
                        m.fs.irrigation_and_drainage.flow_vol_in.fix(flow_scenario)
                        m.fs.reverse_osmosis.feed.pressure.unfix()
                        m.fs.reverse_osmosis.membrane_area.unfix()

                        print("adjusted baseline flow -->", flow_scenario)
                        wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                        if n == 1: s_name = "1p5_mgd"
                        if n == 2: s_name = "3_mgd"
                        if n == 3: s_name = "5_mgd"
                        df_no2 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)                
                        n = n + 1
                        
            
            if case_study == "lithium":
                ### double cap
                if scenario == "baseline": 
                    s_name = "double_cap"
                else:
                    s_name = scenario + "_double_cap"
                print("goes in")
                flow_in_x2 = m.fs.lithium_brine.flow_vol_in[0]() * 2
                m.fs.lithium_brine.flow_vol_in.fix(flow_in_x2)
                print("adjusted baseline flow -->", flow_in_x2)
                wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")
                df_no3 = wt.get_results_table(m=m, case_study=m.fs.train["case_study"], scenario=s_name)

            
            dwi_list = ["emwd", "big_spring"]   
            if m.fs.train["case_study"] in dwi_list:
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "deep_well_injection":
                        wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)

            if scenario in ["edr_ph_ro", "ro_and_mf"]:
                print("goes in")
                wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                    case_study = case_study, skip_small_sens = skip_small_sens)                

            if source_water == "power":
                for key in m.fs.pfd_dict.keys():
                    if m.fs.pfd_dict[key]["Unit"] == "evaporation_pond":
                        wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, 
                                            case_study = case_study, skip_small_sens = skip_small_sens)  



------------------------------------------------------
------------------------------------------------------
----------- iron_and_steel baseline -----------------
------------------------------------------------------
------------------------------------------------------
iron_and_steel

------- Adding Unit Processes -------
well_field
raw_water_pump1
primary_separator
coagulant_addition
coag_and_floc
caustic_soda_addition
dissolved_air_flotation
treated_storage
discharge
tramp_oil_tank
raw_water_pump1_2
buffer_tank
caustic_soda_addition_2
lime_addition_2
aeration_tank
coagulant_addition_2
lime_addition_3
clarifier
treated_storage_2
discharge_2
landfill
-------------------------------------

adding source
adding source
adding source
adding source
[['primary_separator', 'outlet']]
{'coagulant_addition': 0.83, 'raw_water_pump1_2': 0.17}
mixer1
['primary_separator', 'outlet', 'mixer1', 'inlet2']
{'coagulant_addition': 0.83, 'raw_water_pump1_2': 0.17}
mixer2
['primary_separator', 'outlet'


Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.8650000000000001 LCOW --> 0.21800944695309588
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.8800000000000001 LCOW --> 0.2142933768345772
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.8950000000000001 LCOW --> 0.21070186772561783
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.910000

Weighted Average Cost of Capital 5-10% 0.09500000000000004 LCOW --> 0.20636234604180173
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.09750000000000004 LCOW --> 0.20939636192551425
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.10000000000000005 LCOW --> 0.21244852707591055
------- RESET -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.18857817161442797
[1.5, 0, 3.0, 0]
------- Inlet TDS +-25% -------
---------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 0.04977000000000001 LCOW --> 0.19613422106978318
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 0.05115250000000001 LCOW --> 0.19411817979761326
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 0.05253500000000001 LCOW --> 0.1921908597047687
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +-20% 0.053917500000000014 LCOW --> 0.1903460624092298
---------------------------------------------------


Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 39.0 LCOW --> 0.18215487991393986
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 40.5 LCOW --> 0.18149427182481234
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 42.0 LCOW --> 0.1809119685168684
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 43.5 LCOW --> 0.18039806261120364
--------------------------------------------------------

...


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 1.3395000000000001 LCOW --> 6.320757596712815
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 1.3680000000000003 LCOW --> 6.323013380415937
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 1.3965000000000005 LCOW --> 6.325275608638388
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-25% 1.4250000000000005 LCOW --> 6.327543920004706
------- RESET -------
--------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 22.5 LCOW --> 6.888565707635865
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 24.0 LCOW --> 6.7388994433965355
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 25.5 LCOW --> 6.608672437038208
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 27.0 LCOW --> 6.494615320043397
----------------------------------------------------------------------

Degrees of Fr


WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06267499999999995 LCOW --> 6.39504910468051
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06430999999999995 LCOW --> 6.413019402345111
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06594499999999995 LCOW --> 6.430989700009886
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06757999999999995 LCOW --> 6.448959997675282
---------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 20.40988118301805 LCOW --> 6.53666577764027
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 20.649997432230027 LCOW --> 6.521169779692458
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 20.890113681442003 LCOW --> 6.505878156563999
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 21.13022993065398 LCOW --> 6.490608571415947
----------------------------------------------------------------------

Degrees of Freedom: 0

Wa


WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.34999999999999987 LCOW --> 6.3606348736049485
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.3699999999999999 LCOW --> 6.371722325058019
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.3899999999999999 LCOW --> 6.3828097765075515
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.4099999999999999 LCOW --> 6.393897227957723
---------------

In [17]:
m.fs.seawater.flow_vol_in[0]()

1.92775

In [18]:
m.fs.seawater.conc_mass_in[0, "tds"]()

30.0

In [19]:
m.fs.seawater.conc_mass_in[0, "tds"].fix(30.0*0.75)

In [20]:
wt.run_water_tap(m=m, objective=True, skip_small=skip_small_base, print_model_results="summary")

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution maxIterations 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


SW ONSHORE INTAKE:

	total cap investment: 10.21687
	cat and chem cost: 0.0
	electricity cost: 0.35741
	total fixed op cost: 0.13554

SULFURIC ACID ADDITION:

	total cap investment: 0.23073
	cat and chem cost: 0.07937
	electricity cost: 0.0
	total fixed op cost: 0.00307

FERRIC CHLORIDE ADDITION:

	total cap investment: 2.95364
	cat and chem cost: 1.16399
	electricity cost: 1e-05
	total fixed op cost: 0.03933

CHLORINATION:

	total cap investment: 9.26588
	cat and chem cost: 0.16131
	electricity cost: 2e-05
	total fixed op cost: 0.12338

STATIC MIXER:

	total cap investment: 0.11622
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00154

TRI MEDIA FILTRATION:

	total cap investment: 7.60919
	cat and chem cost: 0.0
	electricity c

In [ ]:
	Feed pressure for ro_first_pass: 76.76 bar
	Membrane area for ro_first_pass: 105745.17 m2
	Pure Water Flux for ro_first_pass: 36.5869149842172 lmh
	A constant for ro_first_pass: 4.2
	B constant for ro_first_pass: 0.35
	Feed pressure for ro_second_pass: 42.26 bar
	Membrane area for ro_second_pass: 64751.24 m2
	Pure Water Flux for ro_second_pass: 19.77366442256369 lmh
	A constant for ro_second_pass: 4.2
	B constant for ro_second_pass: 0.35

In [ ]:
	Feed pressure for ro_first_pass: 74.94 bar
	Membrane area for ro_first_pass: 119160.03 m2
	Pure Water Flux for ro_first_pass: 32.8022149171156 lmh
	A constant for ro_first_pass: 4.2
	B constant for ro_first_pass: 0.35
	Feed pressure for ro_second_pass: 30.54 bar
	Membrane area for ro_second_pass: 74315.39 m2
	Pure Water Flux for ro_second_pass: 34.75399096474025 lmh
	A constant for ro_second_pass: 4.2
	B constant for ro_second_pass: 0.35

In [21]:
m.fs.splitter1.display()

Block fs.splitter1

  Variables:
    flow_vol_outlet1 : Volumetric flowrate of water out of unit
        Size=1, Index=fs.time
        Key : Lower : Value              : Upper : Fixed : Stale : Domain
        0.0 : 1e-09 : 0.4099134152969156 : 100.0 : False : False : NonNegativeReals
    conc_mass_outlet1 : Mass concentration of species at outlet
        Size=11, Index=fs.splitter1.conc_mass_outlet1_index
        Key                : Lower : Value                 : Upper : Fixed : Stale : Domain
            (0.0, 'boron') :  None :  0.005852532411820729 :  None : False : False :  Reals
          (0.0, 'bromide') :  None : 0.0007510960780225815 :  None : False : False :  Reals
          (0.0, 'calcium') :  None :  0.004640255769778638 :  None : False : False :  Reals
         (0.0, 'chloride') :  None :   0.21708149900203372 :  None : False : False :  Reals
        (0.0, 'magnesium') :  None :  0.014478141939536731 :  None : False : False :  Reals
        (0.0, 'potassium') :  None :  0

In [ ]:
flow_vol_outlet1 0.4099134152969156 #0.73 for outlet 1
flow_vol_outlet2 0.8322484492391917 #0.36 for outlet 2

In [22]:
0.8322484492391917 / (0.8322484492391917 +0.4099134152969156 )

0.6699999999999998

In [23]:
m.fs.ro_second_pass.flow_vol_in[0]()

0.40991341529888103

In [ ]:
flow_vol_outlet1 0.7311518099723392

In [ ]:
flow_vol_outlet2 0.36011954819533126

In [7]:
0.7311518099723392 / (0.36011954819533126 + 0.7311518099723392)

0.6699999999999999

In [4]:
recovery = m.fs.reverse_osmosis.flow_vol_out[0]()/m.fs.reverse_osmosis.flow_vol_in[0]()
recovery2 = m.fs.ro_stage.flow_vol_out[0]()/m.fs.ro_stage.flow_vol_in[0]()
recovery3 = m.fs.reverse_osmosis_2.flow_vol_out[0]()/m.fs.reverse_osmosis_2.flow_vol_in[0]()

recovery

0.568352132710339

In [5]:
recovery2

0.7288811390942325

In [6]:
recovery3

0.5050004295470119

In [8]:
m.fs.splitter1.display()

Block fs.splitter1

  Variables:
    flow_vol_outlet1 : Volumetric flowrate of water out of unit
        Size=1, Index=fs.time
        Key : Lower : Value             : Upper : Fixed : Stale : Domain
        0.0 : 1e-09 : 0.821360036601861 : 100.0 : False : False : NonNegativeReals
    conc_mass_outlet1 : Mass concentration of species at outlet
        Size=11, Index=fs.splitter1.conc_mass_outlet1_index
        Key                : Lower : Value                  : Upper : Fixed : Stale : Domain
            (0.0, 'boron') :  None :   0.005930116887606596 :  None : False : False :  Reals
          (0.0, 'bromide') :  None :   0.000761053031932074 :  None : False : False :  Reals
          (0.0, 'calcium') :  None :   0.004701769621574556 :  None : False : False :  Reals
         (0.0, 'chloride') :  None :     0.2199592539835969 :  None : False : False :  Reals
        (0.0, 'magnesium') :  None :    0.01467007236788649 :  None : False : False :  Reals
        (0.0, 'potassium') :  None 

In [16]:
wt.display.show_train2(model_name=m)

In [ ]:
#wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)

In [18]:
m.fs.splitter1.flow_vol_in[0]()

0.057347809908287334

In [ ]:
0.006308259089911607

In [ ]:
0.051039550818375724

In [19]:
0.051039550818375724/0.057347809908287334

0.8899999999999999

In [17]:
m.fs.splitter1.display()

Block fs.splitter1

  Variables:
    flow_vol_outlet1 : Volumetric flowrate of water out of unit
        Size=1, Index=fs.time
        Key : Lower : Value                : Upper : Fixed : Stale : Domain
        0.0 : 1e-09 : 0.006308259089911607 : 100.0 : False : False : NonNegativeReals
    conc_mass_outlet1 : Mass concentration of species at outlet
        Size=13, Index=fs.splitter1.conc_mass_outlet1_index
        Key                              : Lower : Value                  : Upper : Fixed : Stale : Domain
            (0.0, 'alkalinity_as_caco3') :  None :    0.11705330011041193 :  None : False : False :  Reals
                        (0.0, 'calcium') :  None : 0.00035160174219506387 :  None : False : False :  Reals
                       (0.0, 'chloride') :  None :  0.0003039890062728146 :  None : False : False :  Reals
        (0.0, 'electrical_conductivity') :  None :     261.97643358044576 :  None : False : False :  Reals
                      (0.0, 'magnesium') :  None :  

In [21]:
m.fs.ozone.flow_vol_in[0]() * 15850.372483753

1169.8873365137872

In [22]:
0.051039550818375724 * 15850.372483753

808.9958918746955

In [6]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))

None
INFO: CONSTR fs.ro_second_pass.component_mass_balance[0.0,bromide]:
    0.00030523928397560113 =/= 0.0
INFO: CONSTR fs.ro_second_pass.component_mass_balance[0.0,calcium]:
    0.0018858015019106475 =/= 0.0
INFO: CONSTR fs.ro_second_pass.component_mass_balance[0.0,chloride]:
    0.08821241304730497 =/= 0.0
INFO: CONSTR fs.ro_second_pass.component_mass_balance[0.0,magnesium]:
    0.0058839593882765604 =/= 0.0
INFO: CONSTR fs.ro_second_pass.component_mass_balance[0.0,potassium]:
    0.0018199477249093575 =/= 0.0
INFO: CONSTR fs.ro_second_pass.component_mass_balance[0.0,sodium]:
    0.049162087155165235 =/= 0.0
INFO: CONSTR fs.ro_second_pass.component_mass_balance[0.0,strontium]:
    5.984962166539979e-06 =/= 0.0
INFO: CONSTR fs.ro_second_pass.component_mass_balance[0.0,sulfate]:
    0.012338632324161034 =/= 0.0
INFO: CONSTR fs.ro_second_pass.bromide_eq: 0.0003052621074764514 =/= 0.0
INFO: CONSTR fs.ro_second_pass.calcium_eq: 0.0018859028784195275 =/= 0.0
INFO: CONSTR fs.ro_second_pass